In [1]:
import cv2
import json
import requests
from os import scandir, getcwd, rename

In [2]:
from os import scandir, getcwd, rename
# Obtener archivos de una ruta
ruta = 'imgAnalyze/'
files = [ruta + '/' + arch.name for arch in scandir(ruta) if arch.is_file()]
# files_lista = []
# for i in files
# files_lista.append(ruta+'\\'+i)

print("Terminó. {}".format(len(files)))

Terminó. 3


In [3]:
# Consulta al servicio azure
vision_base_url = "https://workshopbcrpcomputervision.cognitiveservices.azure.com/vision/v2.0/"
features = "Objects, Description"
language = "en"
analyze_url = "{}analyze?visualFeatures={}&language={}".format(vision_base_url, features, language)
subscription_key = "4168b5554d6b442786c5e53ad83ccd52"

#jsons
jsons = []

# Recorriendo la lista de archivos en la ruta
for i in files[:]:
    # ruta por archivo
    print("Reading {}".format(i))
    image = open(i,"rb")
    d = image.read()
    
    headers = {'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(analyze_url, headers=headers, data=d)
    print(response.text)
    
    obj = json.loads(response.text)
    jsons.append(obj)
    image.close()
print("Fin consulta Azure : D")

Reading imgAnalyze//abbey.png
{"description":{"tags":["outdoor","person","road","woman","sidewalk","street","walking","people","man","city","suit","standing","holding","group","talking","busy","young","wearing","stop","parked","parking","traffic","bus"],"captions":[{"text":"a group of people walking down the street","confidence":0.96979892685936964}]},"objects":[{"rectangle":{"x":54,"y":460,"w":57,"h":37},"object":"footwear","confidence":0.51},{"rectangle":{"x":496,"y":475,"w":53,"h":30},"object":"footwear","confidence":0.5},{"rectangle":{"x":171,"y":87,"w":154,"h":126},"object":"Van","confidence":0.672,"parent":{"object":"car","confidence":0.824,"parent":{"object":"Land vehicle","confidence":0.857,"parent":{"object":"Vehicle","confidence":0.857}}}},{"rectangle":{"x":58,"y":106,"w":191,"h":392},"object":"person","confidence":0.822},{"rectangle":{"x":298,"y":106,"w":177,"h":381},"object":"person","confidence":0.784},{"rectangle":{"x":498,"y":120,"w":211,"h":384},"object":"person","confi

In [4]:
def rect_to_mmxy(rect):
    x = int(rect['x'])
    y = int(rect['y'])
    w = int(rect['w'])
    h = int(rect['h'])
    
    return (x,y),(x+w,y+h)

In [6]:
for i, ruta in enumerate(files[:]):
    j=0
    
    print('Para: {}'.format(ruta))
    image = cv2.imread(ruta)
    
    for line in jsons[i]['objects']:
        # Dibujando cuadros, texto y confiabilidad por objeto detectado
        j+=1
        min_xy, max_xy = rect_to_mmxy(line['rectangle'])
        cv2.rectangle(image, min_xy, max_xy,(255,0,0),2)
        cv2.putText(image, 'obj:{} conf:{}'.format(line['object'], line['confidence']), (min_xy[0],min_xy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
    
    # Mostrando imagen modificada
    cv2.imshow('image',image)
    cv2.waitKey(0)
    # cv2.imwrite('res{}.png'.format(i),image)
    cv2.destroyAllWindows()

Para: imgAnalyze//abbey.png
Para: imgAnalyze//rand.png
Para: imgAnalyze//termi.jpg


In [27]:
for i, ruta in enumerate(files_lista[:]):
    j=0
    print('Para: {}'.format(ruta))
    image = cv2.imread(ruta)
    
    for line in jsons[i]['description']['captions']:
        # Mostrando descripciones por imagen analizada
        print('Text:{} conf:{}'.format(line['text'], line['confidence']))
        j+=1
        min_xy = (20, 20)
        cv2.putText(image, 't:{} c:{}'.format(line['text'], line['confidence']), (min_xy[0],min_xy[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)        
    
    cv2.imshow('image',image)
    cv2.waitKey(0)
    # cv2.imwrite('res{}.png'.format(i),image)
    cv2.destroyAllWindows()

Para: D:\GyS\proyectos\workshopBCRP\notebooks\imgAnalyze\abbey.png
Text:a group of people walking down the street conf:0.9697989268593696
Para: D:\GyS\proyectos\workshopBCRP\notebooks\imgAnalyze\rand.png
Text:a man and a dog walking on a sidewalk conf:0.9246257035076508
Para: D:\GyS\proyectos\workshopBCRP\notebooks\imgAnalyze\termi.jpg
Text:a man riding on the back of a motorcycle conf:0.8181732157819246
